In [1]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
!pip install vecstack

from vecstack import stacking
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings("ignore")

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=bb239a8ba43944bc215612dfac61e026e00249d59c353c9d4687f5c49b8ee6bd
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window

In [4]:
trainfile = r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/Preprocess_Train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/Preprocess_Test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

print(trainData.head())
print(testData.head())


   Unnamed: 0  1stFlrSF  2ndFlrSF  3SsnPorch  BedroomAbvGr  BsmtFinSF1  BsmtFinSF2  BsmtFullBath  BsmtHalfBath  BsmtUnfSF  EnclosedPorch  Fireplaces  FullBath  GarageArea  GarageCars  GarageYrBlt  GrLivArea  HalfBath  Id  KitchenAbvGr  LotArea  LotFrontage  LowQualFinSF  MSSubClass  MasVnrArea  MiscVal  MoSold  OpenPorchSF  OverallCond  OverallQual  PoolArea  ScreenPorch  TotRmsAbvGrd  TotalBsmtSF  WoodDeckSF  YearBuilt  YearRemodAdd  YrSold  BldgType_1Fam  BldgType_2fmCon  BldgType_Duplex  BldgType_Twnhs  BldgType_TwnhsE  BsmtCond_Fa  BsmtCond_Gd  BsmtCond_Po  BsmtCond_TA  BsmtExposure_Av  BsmtExposure_Gd  BsmtExposure_Mn  BsmtExposure_No  BsmtFinType1_ALQ  BsmtFinType1_BLQ  BsmtFinType1_GLQ  BsmtFinType1_LwQ  BsmtFinType1_Rec  BsmtFinType1_Unf  BsmtFinType2_ALQ  BsmtFinType2_BLQ  BsmtFinType2_GLQ  BsmtFinType2_LwQ  BsmtFinType2_Rec  BsmtFinType2_Unf  BsmtQual_Ex  BsmtQual_Fa  BsmtQual_Gd  BsmtQual_TA  CentralAir_N  CentralAir_Y  Condition1_Artery  Condition1_Feedr  Condition1_Norm  \

In [5]:
#Extract training and test data
y_train = trainData["SalePrice"]
X_train = trainData.drop(["SalePrice"], axis=1) #extracting training data without the target column
X_test = testData.copy() #extracting test data without the target column

print(y_train.shape)
print(X_test.shape)

(1460,)
(1459, 272)


In [6]:
X_test.tail()

,Unnamed: 0,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,Id,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MSSubClass,MasVnrArea,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,CentralAir_N,CentralAir_Y,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,PavedDrive_N,PavedDrive_P,PavedDrive_Y,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,SaleCondition_Abnorml,Sa

In [7]:
#Decision Tree Regressor ========================================================================

#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)
clf_predict_Train=clf.predict(X_train)

mean_squared_error(y_train,clf_predict_Train)
print("RMSE (training) for Decision Tree Regressor:{0:10f}".format(mean_squared_error(y_train,clf_predict_Train)))
clf_predict_Test=clf.predict(X_test)

#Hyperparameter tuning done
parameters1 ={'min_samples_split' : range(100,1000,100),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters1 ,n_iter=30,scoring = 'neg_mean_squared_error')
clf_random.fit(X_train, y_train)
grid_parm1=clf_random.best_params_
print(grid_parm1)

#Construct best Decision Tree
clf1 = DecisionTreeRegressor(**grid_parm1)
clf1.fit(X_train,y_train)
clf1_predict_Test = clf1.predict(X_test)

#run cross-validation on best hyperparameters
clf1_cv_score = cross_val_score(clf1, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
print("=== All RMSE ===")
print(clf1_cv_score)
print('\n')
print("=== Mean RMSE ===")
print("Mean RMSE - Decision Tree Regressor: ",clf1_cv_score.mean())

#Save predictions
df_clf = pd.DataFrame()
df_clf['TARGET']=clf_predict_Test
export_csv = df_clf.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/DT_Test.csv')

RMSE (training) for Decision Tree Regressor:  0.000000
{'min_samples_split': 100, 'max_depth': 7}
=== All RMSE ===
[-1.20648344e+09 -1.38679564e+09 -1.05097999e+09 -1.61565763e+09
 -2.51121353e+09 -1.31920348e+09 -1.35321699e+09 -1.09619897e+09
 -2.72459856e+09 -1.50133974e+09]


=== Mean RMSE ===
Mean RMSE - Decision Tree Regressor:  -1576568796.8029091


In [8]:
#Random Forest Regressor==============================================================================

rfc = RandomForestRegressor()
rfc.fit(X_train, y_train)
rfc_predict_Train=rfc.predict(X_train)

mean_squared_error(y_train,rfc_predict_Train)
print("RMSE (training) for Random Forest Regressor:{0:10f}".format(mean_squared_error(y_train,rfc_predict_Train)))
rfc_predict_Test=rfc.predict(X_test)

#Hyperparameter tuning done 
parameters2={'n_estimators':range(10,20,1), 'min_samples_split' : range(10,30,10),'max_depth': range(1,5,2)}
rfc_random = RandomizedSearchCV(rfc,parameters2,n_iter=30,scoring = 'neg_mean_squared_error')
rfc_random.fit(X_train, y_train)
grid_parm2=rfc_random.best_params_
print(grid_parm2)

#Using the parameters obtained from HyperParameterTuning 
rfc1 = RandomForestRegressor(**grid_parm2)
rfc1.fit(X_train,y_train)
rfc1_predict_Test = rfc1.predict(X_test)

#run cross-validation on best hyperparameters
rfc_cv_score = cross_val_score(rfc1, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
print("=== All RMSE Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean RMSE Score ===")
print("Mean RMSE Score - RF Regressor ",rfc_cv_score.mean())

#Save predictions
df_rfc=pd.DataFrame()
df_rfc['TARGET']=rfc1_predict_Test
export_csv = df_rfc.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/RF_Test.csv')

RMSE (training) for Random Forest Regressor:172101747.014603
{'n_estimators': 17, 'min_samples_split': 20, 'max_depth': 3}
=== All RMSE Scores ===
[-1.34997251e+09 -1.38194935e+09 -1.07215977e+09 -2.10358998e+09
 -2.12100524e+09 -1.54286330e+09 -1.33116189e+09 -1.36404035e+09
 -2.40291167e+09 -1.37678637e+09]


=== Mean RMSE Score ===
Mean RMSE Score - RF Regressor  -1604644042.3880973


In [9]:
#Gradient Boosting Regressor================================================================================

gbr =GradientBoostingRegressor()
gbr.fit(X_train, y_train)
gbr_predict_Train=gbr.predict(X_train)

mean_squared_error(y_train,gbr_predict_Train)
print("RMSE (training) for Gradient Boosting Regressor:{0:10f}".format(mean_squared_error(y_train,gbr_predict_Train)))
gbr_predict_Test=gbr.predict(X_test)

#Hyperparameter tuning 

parameters4 = {'n_estimators': [100,200,300], 'learning_rate': [0.1, 0.01, 0.001]} 
gbr_random = RandomizedSearchCV(gbr,parameters4,n_iter=30,scoring = 'neg_mean_squared_error')
gbr_random.fit(X_train, y_train)
grid_parm4=gbr_random.best_params_
print(grid_parm4)

#Using the parameters obtained from HyperParameterTuning 
gbr1 = GradientBoostingRegressor(**grid_parm4)
gbr1.fit(X_train,y_train)
gbr1_predict_Test = gbr1.predict(X_test)

#run cross-validation on best hyperparameters
gbr_cv_score = cross_val_score(gbr1, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
print("=== All RMSE Scores ===")
print(gbr_cv_score)
print('\n')
print("=== Mean RMSE Score ===")
print("Mean RMSE Score - GB Regressor ",gbr_cv_score.mean())

#Save predictions
df_gbr=pd.DataFrame()
df_gbr['TARGET']=gbr_predict_Test
export_csv = df_gbr.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/GBR_Test.csv')

RMSE (training) for Gradient Boosting Regressor:210362028.600837
{'n_estimators': 100, 'learning_rate': 0.1}
=== All RMSE Scores ===
[-4.62674066e+08 -6.48427818e+08 -4.17604437e+08 -1.43843985e+09
 -8.78780968e+08 -6.67942440e+08 -7.42462861e+08 -4.43319168e+08
 -7.45324276e+08 -6.20005432e+08]


=== Mean RMSE Score ===
Mean RMSE Score - GB Regressor  -706498131.3936914


In [10]:
df_gbr.shape

(1459, 1)

In [11]:
#SVR regressor==============================================================================

svc = LinearSVR()
svc.fit(X_train, y_train)
svc_predict_Train=svc.predict(X_train)

mean_squared_error(y_train,svc_predict_Train)
print("RMSE (training) for GB Regressor:{0:10f}".format(mean_squared_error(y_train,svc_predict_Train)))
svc_predict_Test=svc.predict(X_test)

#Hyperparameter tuning done 
parameters3 = {'C': [0.1, 1, 10, 100, 1000], 'tol': [1, 0.1, 0.01, 0.001, 0.0001]} 
svc_random = RandomizedSearchCV(svc,parameters3,n_iter=30,scoring = 'neg_mean_squared_error')
svc_random.fit(X_train, y_train)
grid_parm3=svc_random.best_params_
print(grid_parm3)

#Using the parameters obtained from HyperParameterTuning 
svc1 = LinearSVR(**grid_parm3)
svc1.fit(X_train,y_train)
svc1_predict_Test = svc1.predict(X_test)

#run cross-validation on best hyperparameters
svc_cv_score = cross_val_score(svc1, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
print("=== All RMSE square Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean RMSE Score ===")
print("Mean RMSE Score - SVM Regressor ",svc_cv_score.mean())

#Save predictions
df_svc=pd.DataFrame()
df_svc['TARGET']=svc_predict_Test
export_csv = df_rfc.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/LinearSVC_Test.csv')

RMSE (training) for GB Regressor:1708572680.842394
{'tol': 0.01, 'C': 1}
=== All RMSE square Scores ===
[-1.18495491e+09 -1.90340901e+09 -1.82528150e+09 -2.60436328e+09
 -3.67615587e+09 -3.31145046e+09 -1.44787696e+09 -1.76196487e+10
 -7.82299050e+09 -2.70804325e+09]


=== Mean RMSE Score ===
Mean RMSE Score - SVM Regressor  -4410417440.519708


In [12]:
#MLP Regressor================================================================================

mlp =MLPRegressor()
mlp.fit(X_train, y_train)
mlp_predict_Train=mlp.predict(X_train)

mean_squared_error(y_train,mlp_predict_Train)
print("RMSE (training) for MLP Regressor:{0:10f}".format(mean_squared_error(y_train,mlp_predict_Train)))
mlp_predict_Test=mlp.predict(X_test)

#Hyperparameter tuning 
parameters5 = {'alpha':[0.1, 0.01, 0.001, 0.0001]} 
mlp_random = RandomizedSearchCV(gbr,parameters5,n_iter=30,scoring = 'neg_mean_squared_error')
mlp_random.fit(X_train, y_train)
grid_parm5=mlp_random.best_params_
print(grid_parm5)

#Using the parameters obtained from HyperParameterTuning 
mlp1 = MLPRegressor(**grid_parm5)
mlp1.fit(X_train,y_train)
mlp1_predict_Test = mlp1.predict(X_test)

#cross validation
mlp_cv_score = cross_val_score(mlp1, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
print("=== All RMSE Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean RMSE Score ===")
print("Mean RMSE Score - MLP Regressor ",mlp_cv_score.mean())

#Save predictions
df_mlp=pd.DataFrame()
df_mlp['TARGET']=mlp_predict_Test
export_csv = df_mlp.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/MLP_Test.csv')

RMSE (training) for MLP Regressor:1912016282.226880
{'alpha': 0.01}
=== All RMSE Scores ===
[-1.38392003e+09 -1.69280941e+09 -1.66836527e+09 -2.47438009e+09
 -2.67855121e+09 -1.56389546e+09 -1.72341493e+09 -1.58704741e+09
 -5.30138759e+09 -1.41080056e+09]


=== Mean RMSE Score ===
Mean RMSE Score - MLP Regressor  -2148457197.408672


In [13]:
df_mlp.shape

(1459, 1)

In [14]:
#Stochastic Gradient Descent Regressor================================================================================

sgd = SGDRegressor(loss="squared_loss")
sgd.fit(X_train, y_train)
sgd_predict_Train=sgd.predict(X_train)

mean_squared_error(y_train,sgd_predict_Train)
print("RMSE (training) for SGD Regressor:{0:10f}".format(mean_squared_error(y_train,sgd_predict_Train)))
sgd_predict_Test=sgd.predict(X_test)

#Hyperparameter tuning 

parameters6 = {'alpha': [0.99, 0.1, 0.01, 0.001, 0.0001]} 
sgd_random = RandomizedSearchCV(gbr,parameters6,n_iter=30,scoring = 'neg_mean_squared_error')
sgd_random.fit(X_train, y_train)
grid_parm6=sgd_random.best_params_
print(grid_parm6)

#Using the parameters obtained from HyperParameterTuning  
sgd1 = SGDRegressor(**grid_parm6)
sgd1.fit(X_train,y_train)
sgd1_predict_Test = sgd1.predict(X_test)

#run cross-validation on best hyperparameters
sgd_cv_score = cross_val_score(sgd1, X_train, y_train, cv=10, scoring="neg_mean_squared_error")
print("=== All RMSE Scores ===")
print(sgd_cv_score)
print('\n')
print("=== Mean RMSE Score ===")
print("Mean RMSE Score - SGD Regressor ",sgd_cv_score.mean())

#Save predictions
df_sgd=pd.DataFrame()
df_sgd['TARGET']=sgd_predict_Test
export_csv = df_sgd.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/SGD_Test.csv')


RMSE (training) for SGD Regressor:16164906402561287034795787477319680.000000
{'alpha': 0.001}
=== All RMSE Scores ===
[-3.33618505e+35 -1.47226101e+34 -6.87948147e+34 -1.59717706e+34
 -3.07879743e+35 -4.11477536e+33 -2.37817011e+34 -6.65490872e+32
 -1.21538168e+32 -6.36797384e+33]


=== Mean RMSE Score ===
Mean RMSE Score - SGD Regressor  -7.760389228006388e+34


In [15]:
#STACKING MODELS =====================================================================

models = [ DecisionTreeRegressor(**grid_parm1), RandomForestRegressor(**grid_parm2), \
          LinearSVR(**grid_parm3), GradientBoostingRegressor(**grid_parm4), \
          MLPRegressor(**grid_parm5), SGDRegressor(**grid_parm6) ]
      
S_Train, S_Test = stacking(models,                  
                            
                           X_train, y_train, X_test, 

                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2)



___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [6]

model  0:     [DecisionTreeRegressor]
    fold  0:  [25941.85516158]
    fold  1:  [27833.17167446]
    fold  2:  [25328.55537815]
    fold  3:  [27851.65956077]
    ----
    MEAN:     [26738.81044374] + [1124.72404753]
    FULL:     [26738.81044374]

model  1:     [RandomForestRegressor]
    fold  0:  [25918.59929147]
    fold  1:  [28930.97112768]
    fold  2:  [23794.37141625]
    fold  3:  [26201.15704566]
    ----
    MEAN:     [26211.27472026] + [1825.09803870]
    FULL:     [26211.27472026]

model  2:     [LinearSVR]
    fold  0:  [25505.89404186]
    fold  1:  [76698.32625946]
    fold  2:  [26778.70794688]
    fold  3:  [34006.48380585]
    ----
    MEAN:     [40747.35301351] + [2100

In [16]:
print(S_Train.shape, y_train.shape, S_Test.shape)
S_Train = pd.DataFrame(S_Train).copy(deep = True)
S_Test = pd.DataFrame(S_Test).copy(deep = True)

(1460, 6) (1460,) (1459, 6)


In [17]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingRegressor()
model.fit(S_Train, y_train)
model_predict_Train = model.predict(S_Train)

mean_squared_error(y_train,model_predict_Train)
print("RMSE (training) for Stacked Gradient Boosting Regressor:{0:10f}".format(mean_squared_error(y_train,model_predict_Train)))
model_predict_Test=model.predict(S_Test)

#Hyperparameter tuning
parameters = {'n_estimators': [100,200,300], 'learning_rate': [0.1, 0.01, 0.001]} 
model_random = RandomizedSearchCV(model,parameters,n_iter=30,scoring = 'neg_mean_squared_error')
model_random.fit(S_Train, y_train)
grid_parm=model_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning 
model1 = GradientBoostingRegressor(**grid_parm)
model1.fit(S_Train,y_train)
model1_predict_Test = model1.predict(S_Test)

#run cross-validation on best hyperparameters
model_cv_score = cross_val_score(model1, S_Train, y_train, cv=10, scoring="neg_mean_squared_error")
print("=== All RMSE Scores ===")
print(model_cv_score)
print('\n')
print("=== Mean RMSE Score ===")
print("Mean RMSE Score - GB Regressor ",model_cv_score.mean())

#Save predictions
df_model=pd.DataFrame()
df_model['TARGET']=model_predict_Test
export_csv = df_model.to_csv(r'/gdrive/My Drive/CIS_508/Colab Notebooks/TeamAsmt1/StackingModel_Test.csv')



RMSE (training) for Stacked Gradient Boosting Regressor:304730146.182574
{'n_estimators': 300, 'learning_rate': 0.01}
=== All RMSE Scores ===
[-4.78907613e+08 -6.43074624e+08 -4.54520199e+08 -2.21320721e+09
 -1.23548890e+09 -8.87817969e+08 -5.48958685e+08 -4.66798937e+08
 -1.02690694e+09 -6.24136640e+08]


=== Mean RMSE Score ===
Mean RMSE Score - GB Regressor  -857981771.9094647
